In [1]:
import numpy as np
import pandas as pd
import sklearn
import json
from pprint import pprint
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import metrics

from sklearn.model_selection import KFold 
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import FeatureUnion
from scipy.sparse import coo_matrix, hstack, vstack
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer
from sklearn.metrics import f1_score

## Importando DataSet

In [2]:
filename = 'CAvideos.csv'
category_filename = 'CA_category_id.json'

videos = pd.read_csv(filename)
videos = videos.dropna(axis=0)
print("DataSet null inputs: \n"+str(videos.isnull().sum())+"\n")
print(filename+" shape: "+str(videos.shape))
videos.head()

DataSet null inputs: 
video_id                  0
trending_date             0
title                     0
channel_title             0
category_id               0
publish_time              0
tags                      0
views                     0
likes                     0
dislikes                  0
comment_count             0
thumbnail_link            0
comments_disabled         0
ratings_disabled          0
video_error_or_removed    0
description               0
dtype: int64

CAvideos.csv shape: (39585, 16)


,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...
1,0dBIkQ4Mz1M,17.14.11,PLUSH - Bad Unboxing Fan Mail,iDubbbzTV,23,2017-11-13T17:00:00.000Z,"plush|""bad unboxing""|""unboxing""|""fan mail""|""id...",1014651,127794,1688,13030,https://i.ytimg.com/vi/0dBIkQ4Mz1M/default.jpg,False,False,False,STill got a lot of packages. Probably will las...
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146035,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095828,132239,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...
4,2Vv-BfVoq4g,17.14.11,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,10,2017-11-09T11:04:14.000Z,"edsheeran|""ed sheeran""|""acoustic""|""live""|""cove...",33523622,1634130,21082,85067,https://i.ytimg.com/vi/2Vv-BfVoq4g/default.jpg,False,False,False,🎧: https://ad.gt/yt-perfect\n💰: https://atlant...


# Tratamento do DataSet

## Correção de formato de data-hora para padrão Unix

In [3]:
videos['trending_date'] = pd.to_datetime(videos['trending_date'], format='%y.%d.%m')
videos['publish_time'] = pd.to_datetime(videos['publish_time'], format='%Y-%m-%dT%H:%M:%S.%fZ')

# separates date and time into two columns from 'publish_time' column

videos.insert(4, 'publish_date', videos['publish_time'].dt.date)
videos['publish_time'] = videos['publish_time'].dt.time
videos['publish_date']=pd.to_datetime(videos['publish_date'])

videos.head()

,video_id,trending_date,title,channel_title,publish_date,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,n1WpP7iowLc,2017-11-14,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,2017-11-10,10,17:00:03,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...
1,0dBIkQ4Mz1M,2017-11-14,PLUSH - Bad Unboxing Fan Mail,iDubbbzTV,2017-11-13,23,17:00:00,"plush|""bad unboxing""|""unboxing""|""fan mail""|""id...",1014651,127794,1688,13030,https://i.ytimg.com/vi/0dBIkQ4Mz1M/default.jpg,False,False,False,STill got a lot of packages. Probably will las...
2,5qpjK5DgCt4,2017-11-14,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,2017-11-12,23,19:05:24,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146035,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,d380meD0W0M,2017-11-14,I Dare You: GOING BALD!?,nigahiga,2017-11-12,24,18:01:41,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095828,132239,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...
4,2Vv-BfVoq4g,2017-11-14,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,2017-11-09,10,11:04:14,"edsheeran|""ed sheeran""|""acoustic""|""live""|""cove...",33523622,1634130,21082,85067,https://i.ytimg.com/vi/2Vv-BfVoq4g/default.jpg,False,False,False,🎧: https://ad.gt/yt-perfect\n💰: https://atlant...


## Tratando para pegar somente a ultima entrada para cada video (a entrada mais atualizada)

In [4]:
videos_lastentry = videos.sort_values('trending_date', ascending=False).drop_duplicates(['video_id'])
print(filename+" shape: "+str(videos_lastentry.shape))
videos_lastentry.head()

CAvideos.csv shape: (23467, 17)


,video_id,trending_date,title,channel_title,publish_date,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
40880,POTgw38-m58,2018-06-14,【完整版】遇到恐怖情人該怎麼辦？2018.06.13小明星大跟班,我愛小明星大跟班,2018-06-13,24,16:00:03,"吳宗憲|""吳姍儒""|""小明星大跟班""|""Sandy""|""Jacky wu""|""憲哥""|""中天...",107392,300,62,251,https://i.ytimg.com/vi/POTgw38-m58/default.jpg,False,False,False,藝人：李妍瑾、玉兔、班傑、LaLa、小優、少少專家：陳筱屏(律師)、Wendy(心理師)、羅...
40741,F9y3QPJwzTo,2018-06-14,TV版 |《如果，爱》 第31集：张怡荻得知儿子病情 Love Won't Wait EP3...,芒果TV独播剧场 Mango TV Drama Channel,2018-06-13,24,16:38:16,"芒果TV独播剧场|""湖南卫视 电视剧""|""芒果TV 电视剧""|""芒果台 电视剧""|""huna...",100941,222,50,83,https://i.ytimg.com/vi/F9y3QPJwzTo/default.jpg,False,False,False,【 欢迎订阅芒果TV独播剧场：https://goo.gl/AUAfZ7 】本集看点：陆阳拿...
40751,tQQDGtqUAQk,2018-06-14,Adrienne & Israel Houghton's Love Story | All ...,All Things Adrienne,2018-06-12,26,17:00:11,"Adrienne Houghton|""Adrienne Bailon""|""All Thing...",358052,21199,338,3239,https://i.ytimg.com/vi/tQQDGtqUAQk/default.jpg,False,False,False,"Adrienne Houghton and her husband, Israel Houg..."
40750,5vmhVFK5A8Y,2018-06-14,YOUTUBERS REACT TO LIL TAY,FBE,2018-06-12,24,21:45:00,"Lil Tay|""Lil Tay Instagram""|""Lil Tay Reaction""...",766861,23745,993,5092,https://i.ytimg.com/vi/5vmhVFK5A8Y/default.jpg,False,False,False,CLICK TO SUBSCRIBE TO THE YOUTUBERS IN THIS EP...
40749,qQn6TsbYrH8,2018-06-14,BIG SHAQ - MAN DON'T DANCE (OFFICIAL MUSIC VIDEO),Michael Dapaah,2018-06-10,10,21:00:18,"man don't dance|""man dont dance""|""BIG SHAQ - M...",6891734,400420,18159,36025,https://i.ytimg.com/vi/qQn6TsbYrH8/default.jpg,False,False,False,Man Don't Dance performed by Big Shaq (Michael...


## Buscando o nome da categoria pelo identificador e armazenando em nova coluna

In [5]:
with open(category_filename) as f:
    category = json.load(f)

In [6]:
def category_replace(c_id):
    for i in category["items"]:
        if int(c_id) == int(i["id"]):
             return i["snippet"]["title"]
    return "None"

In [7]:
videos_lastentry['category_name'] = videos_lastentry.apply(lambda row: category_replace(row['category_id']), axis=1)
print(filename+" shape: "+str(videos_lastentry.shape))
videos_lastentry.head()

CAvideos.csv shape: (23467, 18)


,video_id,trending_date,title,channel_title,publish_date,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,category_name
40880,POTgw38-m58,2018-06-14,【完整版】遇到恐怖情人該怎麼辦？2018.06.13小明星大跟班,我愛小明星大跟班,2018-06-13,24,16:00:03,"吳宗憲|""吳姍儒""|""小明星大跟班""|""Sandy""|""Jacky wu""|""憲哥""|""中天...",107392,300,62,251,https://i.ytimg.com/vi/POTgw38-m58/default.jpg,False,False,False,藝人：李妍瑾、玉兔、班傑、LaLa、小優、少少專家：陳筱屏(律師)、Wendy(心理師)、羅...,Entertainment
40741,F9y3QPJwzTo,2018-06-14,TV版 |《如果，爱》 第31集：张怡荻得知儿子病情 Love Won't Wait EP3...,芒果TV独播剧场 Mango TV Drama Channel,2018-06-13,24,16:38:16,"芒果TV独播剧场|""湖南卫视 电视剧""|""芒果TV 电视剧""|""芒果台 电视剧""|""huna...",100941,222,50,83,https://i.ytimg.com/vi/F9y3QPJwzTo/default.jpg,False,False,False,【 欢迎订阅芒果TV独播剧场：https://goo.gl/AUAfZ7 】本集看点：陆阳拿...,Entertainment
40751,tQQDGtqUAQk,2018-06-14,Adrienne & Israel Houghton's Love Story | All ...,All Things Adrienne,2018-06-12,26,17:00:11,"Adrienne Houghton|""Adrienne Bailon""|""All Thing...",358052,21199,338,3239,https://i.ytimg.com/vi/tQQDGtqUAQk/default.jpg,False,False,False,"Adrienne Houghton and her husband, Israel Houg...",Howto & Style
40750,5vmhVFK5A8Y,2018-06-14,YOUTUBERS REACT TO LIL TAY,FBE,2018-06-12,24,21:45:00,"Lil Tay|""Lil Tay Instagram""|""Lil Tay Reaction""...",766861,23745,993,5092,https://i.ytimg.com/vi/5vmhVFK5A8Y/default.jpg,False,False,False,CLICK TO SUBSCRIBE TO THE YOUTUBERS IN THIS EP...,Entertainment
40749,qQn6TsbYrH8,2018-06-14,BIG SHAQ - MAN DON'T DANCE (OFFICIAL MUSIC VIDEO),Michael Dapaah,2018-06-10,10,21:00:18,"man don't dance|""man dont dance""|""BIG SHAQ - M...",6891734,400420,18159,36025,https://i.ytimg.com/vi/qQn6TsbYrH8/default.jpg,False,False,False,Man Don't Dance performed by Big Shaq (Michael...,Music


## Transformando categorias em OneHotEncoder

In [8]:
# DESNECESSARIO - UTILIZADO ONEHOTENCODER() NO MAPPER
#videos_lastentry = videos_lastentry.join(pd.get_dummies(videos_lastentry['category_name']))
#videos_lastentry.head()

## Definindo sentimento por video

In [9]:
#positive sentiment = 60% da quantidade de likes+dislikes
videos_lastentry['sentiment'] = videos_lastentry['likes'] >= 0.6*(videos_lastentry['likes']+videos_lastentry['dislikes']) 
videos_lastentry.head()

,video_id,trending_date,title,channel_title,publish_date,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,category_name,sentiment
40880,POTgw38-m58,2018-06-14,【完整版】遇到恐怖情人該怎麼辦？2018.06.13小明星大跟班,我愛小明星大跟班,2018-06-13,24,16:00:03,"吳宗憲|""吳姍儒""|""小明星大跟班""|""Sandy""|""Jacky wu""|""憲哥""|""中天...",107392,300,62,251,https://i.ytimg.com/vi/POTgw38-m58/default.jpg,False,False,False,藝人：李妍瑾、玉兔、班傑、LaLa、小優、少少專家：陳筱屏(律師)、Wendy(心理師)、羅...,Entertainment,True
40741,F9y3QPJwzTo,2018-06-14,TV版 |《如果，爱》 第31集：张怡荻得知儿子病情 Love Won't Wait EP3...,芒果TV独播剧场 Mango TV Drama Channel,2018-06-13,24,16:38:16,"芒果TV独播剧场|""湖南卫视 电视剧""|""芒果TV 电视剧""|""芒果台 电视剧""|""huna...",100941,222,50,83,https://i.ytimg.com/vi/F9y3QPJwzTo/default.jpg,False,False,False,【 欢迎订阅芒果TV独播剧场：https://goo.gl/AUAfZ7 】本集看点：陆阳拿...,Entertainment,True
40751,tQQDGtqUAQk,2018-06-14,Adrienne & Israel Houghton's Love Story | All ...,All Things Adrienne,2018-06-12,26,17:00:11,"Adrienne Houghton|""Adrienne Bailon""|""All Thing...",358052,21199,338,3239,https://i.ytimg.com/vi/tQQDGtqUAQk/default.jpg,False,False,False,"Adrienne Houghton and her husband, Israel Houg...",Howto & Style,True
40750,5vmhVFK5A8Y,2018-06-14,YOUTUBERS REACT TO LIL TAY,FBE,2018-06-12,24,21:45:00,"Lil Tay|""Lil Tay Instagram""|""Lil Tay Reaction""...",766861,23745,993,5092,https://i.ytimg.com/vi/5vmhVFK5A8Y/default.jpg,False,False,False,CLICK TO SUBSCRIBE TO THE YOUTUBERS IN THIS EP...,Entertainment,True
40749,qQn6TsbYrH8,2018-06-14,BIG SHAQ - MAN DON'T DANCE (OFFICIAL MUSIC VIDEO),Michael Dapaah,2018-06-10,10,21:00:18,"man don't dance|""man dont dance""|""BIG SHAQ - M...",6891734,400420,18159,36025,https://i.ytimg.com/vi/qQn6TsbYrH8/default.jpg,False,False,False,Man Don't Dance performed by Big Shaq (Michael...,Music,True


## Normalização das features numéricas

In [10]:
videos_lastentry.describe()

,category_id,views,likes,dislikes,comment_count
count,23467.000000,2.346700e+04,2.346700e+04,2.346700e+04,2.346700e+04
mean,21.194145,8.544501e+05,2.663405e+04,1.314972e+03,3.470985e+03
std,6.597851,2.659206e+06,9.681204e+04,1.275178e+04,1.520341e+04
min,1.000000,1.023000e+03,0.000000e+00,0.000000e+00,0.000000e+00
25%,20.000000,1.189680e+05,1.693500e+03,7.900000e+01,3.190000e+02
50%,24.000000,2.978410e+05,5.940000e+03,2.320000e+02,1.010000e+03
75%,24.000000,7.387775e+05,1.959650e+04,6.960000e+02,2.676000e+03
max,43.000000,1.378431e+08,5.053338e+06,1.602383e+06,1.114800e+06


In [11]:
# Normalização
videos_lastentry['views_nonnormal'] = videos_lastentry['views']
videos_lastentry['views'] = (videos_lastentry['views']-videos_lastentry['views'].min())/(videos_lastentry['views'].max()-videos_lastentry['views'].min())
#videos_lastentry['likes'] = (videos_lastentry['likes']-videos_lastentry['likes'].min())/(videos_lastentry['likes'].max()-videos_lastentry['likes'].min())
#videos_lastentry['dislikes'] = (videos_lastentry['dislikes']-videos_lastentry['dislikes'].min())/(videos_lastentry['dislikes'].max()-videos_lastentry['dislikes'].min())
videos_lastentry['comment_count'] = (videos_lastentry['comment_count']-videos_lastentry['comment_count'].min())/(videos_lastentry['comment_count'].max()-videos_lastentry['comment_count'].min())
videos_lastentry.describe()

,category_id,views,likes,dislikes,comment_count,views_nonnormal
count,23467.000000,23467.000000,2.346700e+04,2.346700e+04,23467.000000,2.346700e+04
mean,21.194145,0.006191,2.663405e+04,1.314972e+03,0.003114,8.544501e+05
std,6.597851,0.019292,9.681204e+04,1.275178e+04,0.013638,2.659206e+06
min,1.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,1.023000e+03
25%,20.000000,0.000856,1.693500e+03,7.900000e+01,0.000286,1.189680e+05
50%,24.000000,0.002153,5.940000e+03,2.320000e+02,0.000906,2.978410e+05
75%,24.000000,0.005352,1.959650e+04,6.960000e+02,0.002400,7.387775e+05
max,43.000000,1.000000,5.053338e+06,1.602383e+06,1.000000,1.378431e+08


## Separando conjuntos de treino, validação e teste

In [12]:
train_data = videos_lastentry.sample(frac=0.8, random_state=200)
test_data = videos_lastentry.drop(train_data.index)
print(train_data.shape, test_data.shape)

(18774, 20) (4693, 20)


# Classificador

## União de features

In [13]:
mapper = DataFrameMapper([
    ('title', CountVectorizer(max_features = 2000)),
    ('description', CountVectorizer(max_features = 2000)),
    ('tags', CountVectorizer(max_features = 2000)),
    ('category_name', LabelBinarizer()),
    ('comments_disabled', None),
    ('views', None),
    ('comment_count', None)
])
x_train = mapper.fit_transform(train_data)
y_train = train_data['sentiment'].values

In [14]:
x_train.shape

(18774, 6020)

In [15]:
classifier = LogisticRegression(penalty = 'l1')
classifier.fit(x_train, y_train)

c:\users\work\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l1', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [16]:
x_test = mapper.transform(test_data)
y_test = test_data['sentiment'].values
predicted = classifier.predict(x_test)

In [17]:
probs = classifier.predict_proba(x_test)
print(probs)

[[2.00677771e-05 9.99979932e-01]
 [1.58154886e-06 9.99998418e-01]
 [4.46355464e-01 5.53644536e-01]
 ...
 [1.48641051e-08 9.99999985e-01]
 [5.17776977e-05 9.99948222e-01]
 [5.48119457e-02 9.45188054e-01]]


In [18]:
print("Acuracia: "+str(metrics.accuracy_score(y_test, predicted)))
print("ROC AUC score: "+str(metrics.roc_auc_score(y_test, probs[:, 1])))

Acuracia: 0.9867888344342638
ROC AUC score: 0.7796270226904622


In [19]:
predicted = classifier.predict(mapper.transform(videos_lastentry))

In [20]:
videos_output = videos_lastentry
videos_output['predicted_sentiment'] = predicted

videos_output = videos_output.sort_values('predicted_sentiment', ascending=False)
videos_output.head()

,video_id,trending_date,title,channel_title,publish_date,category_id,publish_time,tags,views,likes,...,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,category_name,sentiment,views_nonnormal,predicted_sentiment
40880,POTgw38-m58,2018-06-14,【完整版】遇到恐怖情人該怎麼辦？2018.06.13小明星大跟班,我愛小明星大跟班,2018-06-13,24,16:00:03,"吳宗憲|""吳姍儒""|""小明星大跟班""|""Sandy""|""Jacky wu""|""憲哥""|""中天...",0.000772,300,...,0.000225,https://i.ytimg.com/vi/POTgw38-m58/default.jpg,False,False,False,藝人：李妍瑾、玉兔、班傑、LaLa、小優、少少專家：陳筱屏(律師)、Wendy(心理師)、羅...,Entertainment,True,107392,True
14304,eQh5bWkJtz8,2018-01-26,Free iPhone X Case - Buying 5 Wish Tech Items,TechSmartt,2018-01-24,28,23:53:39,"wish|""wish.com""|""wishbusters""|""wish haul""|""buy...",0.002202,33160,...,0.002740,https://i.ytimg.com/vi/eQh5bWkJtz8/default.jpg,False,False,False,I bought 5 knockoff tech and gadget items on t...,Science & Technology,True,304523,True
14307,s3sTcXqGZ7Q,2018-01-26,Top 10 Failed Oscar Bait Movies of 2017,WatchMojo.com,2018-01-23,24,22:00:01,"Top 10|""List""|""movies""|""film""|""oscar""|""bad""|""w...",0.003176,5289,...,0.001606,https://i.ytimg.com/vi/s3sTcXqGZ7Q/default.jpg,False,False,False,Top 10 Failed Oscar Bait Movies of 2017\nSubsc...,Entertainment,True,438756,True
14308,1cpb7XIu9fQ,2018-01-26,Between the Games - High Intensity Snacking,Achievement Hunter,2018-01-24,20,18:00:00,"Achievement Hunter|""AH""|""video game""|""gaming""|...",0.001729,6781,...,0.000633,https://i.ytimg.com/vi/1cpb7XIu9fQ/default.jpg,False,False,False,This new game is sweet. All you need is some b...,Gaming,True,239406,True
14310,ATOu109je7A,2018-01-26,Did You Get My Email?,CollegeHumor,2018-01-23,23,18:30:01,"Collegehumor|""CH originals""|""comedy""|""sketch c...",0.005147,27071,...,0.001054,https://i.ytimg.com/vi/ATOu109je7A/default.jpg,False,False,False,"I was going to respond, as soon as I finished ...",Comedy,True,710481,True


In [21]:
f1_score(videos_output['sentiment'].astype(int), videos_output['predicted_sentiment'].astype(int), average='macro')

0.7385187401951862

In [22]:
get_n_features = 10
print("The "+str(get_n_features)+" most positive-weighted words are: ")
for feature_id in sorted(range(len(classifier.coef_[0])), key=lambda i: classifier.coef_[0][i], reverse=True)[:get_n_features]:
    print("%s (%f)" % (mapper.transformed_names_[feature_id], classifier.coef_[0][feature_id]))
print()
print("The "+str(get_n_features)+" most negative-weighted words are: ")
for feature_id in sorted(range(len(classifier.coef_[0])), key=lambda i: classifier.coef_[0][i], reverse=False)[:get_n_features]:
    print("%s (%f)" % (mapper.transformed_names_[feature_id], classifier.coef_[0][feature_id]))

The 10 most positive-weighted words are: 
description_visit (1.849680)
tags_chris (1.772648)
description_18 (1.376542)
title_music (1.297277)
description_need (1.248896)
description_personal (1.216229)
description_ni (1.187350)
description_find (1.186345)
description_business (1.175519)
tags_philippines (1.089423)

The 10 most negative-weighted words are: 
tags_variety (-2.651930)
title_media (-2.077802)
title_rockets (-1.997487)
tags_brazil (-1.832721)
description_matters (-1.795315)
description_feel (-1.557807)
tags_rapper (-1.531184)
description_pinterest (-1.488185)
description_wrong (-1.368048)
title_talks (-1.350268)


# K-Fold

In [23]:
def run_kfold(clf,X,Y):
    kf = KFold(n_splits=10)
    outcomes = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        clf.fit(X_train, y_train)
        predictions = clf.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        outcomes.append(accuracy)
    mean_outcome = np.mean(outcomes)
    print("Mean Accuracy: {0}".format(mean_outcome)) 

In [24]:
run_kfold(classifier,x_train,y_train)

c:\users\work\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\work\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\work\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\work\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\work\appdata\local\programs\python\python37\lib\site-pa

Mean Accuracy: 0.9837543538932984


In [25]:
videos_lastentry['like_ratio'] = videos_lastentry['likes']/(videos_lastentry['likes']+videos_lastentry['dislikes'])

In [26]:
from IPython.display import HTML, display

# We choose the 10 most trending videos
selected_columns = ['title', 'channel_title', 'thumbnail_link', 'publish_date', 'category_name', 'likes', 'dislikes', 'views_nonnormal', 'like_ratio']

most_frequent = videos_lastentry.groupby(selected_columns)['video_id'].agg(
    {"code_count": len}).sort_values(by=['like_ratio']
).head(10).reset_index()

# Construction of HTML table with miniature photos assigned to the most popular movies
table_content = ''
max_title_length = 50

for date, row in most_frequent.T.iteritems():
    HTML_row = '<tr>'
    HTML_row += '<td><img src="' + str(row[2]) + '"style="width:100px;height:100px;"></td>'
    HTML_row += '<td>' + str(row[1]) + '</td>'
    HTML_row += '<td>' + str(row[0])  + '</td>'
    HTML_row += '<td>' + str(row[4]) + '</td>'
    HTML_row += '<td>' + str(row[3]) + '</td>'
    HTML_row += '<td>' + str(row[5]) + '</td>'
    HTML_row += '<td>' + str(row[6]) + '</td>'
    HTML_row += '<td>' + str(row[7]) + '</td>'
    
    table_content += HTML_row + '</tr>'

display(HTML(
    '<table><tr><th>Photo</th><th>Channel Name</th><th style="width:250px;">Title</th><th>Category</th><th>Publish Date</th><th>Likes</th><th>Dislikes</th><th>Views</th></tr>{}</table>'.format(table_content))
)

c:\users\work\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  import sys


Photo,Channel Name,Title,Category,Publish Date,Likes,Dislikes,Views
,Daily Caller,PSA from Chairman of the FCC Ajit Pai,People & Blogs,2017-12-13 00:00:00,7955,190227,961961
,Washington Post,The FCC repeals its net neutrality rules,News & Politics,2017-12-14 00:00:00,5707,126649,1192532
,Wochit News,Cirque Du Soleil Performer Falls To His Death During Performance,News & Politics,2018-03-19 00:00:00,33,554,198215
,TIME,FCC Has Voted To Repeal Net Neutrality | TIME,News & Politics,2017-12-14 00:00:00,1288,17657,349669
,Roy Moore for Senate,Judge Roy Moore Campaign Statement,People & Blogs,2017-12-14 00:00:00,3961,54063,708519
,ReBoot: The Guardian Code,Reboot: The Guardian Code Official Trailer,Film & Animation,2018-02-22 00:00:00,569,7329,212306
,lHeadHunterl,How To Make Money Online With Mobile/Tablet ( Make Money By Installing Apps ),People & Blogs,2017-11-10 00:00:00,90,1022,12603
,Closer,JO 2018 : Oups ! La patineuse Gabriella Papadakis dévoile un téton lors de sa performance,People & Blogs,2018-02-19 00:00:00,140,1047,220550
,Newsy,A Wardrobe Malfunction Might Cost This Team An Olympic Gold Medal,Sports,2018-02-19 00:00:00,15,108,61283
,BBC News,Justin Trudeau: 'Peoplekind' not 'mankind'- BBC News,News & Politics,2018-02-07 00:00:00,171,1123,32534


In [27]:
videos_lastentry.loc[videos_lastentry['description'].str.contains("andretti")]

,video_id,trending_date,title,channel_title,publish_date,category_id,publish_time,tags,views,likes,...,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,category_name,sentiment,views_nonnormal,predicted_sentiment,like_ratio
